In [1]:
import numpy
import pandas
import io
import uuid
import psycopg.sql
import pyarrow
import pyarrow.parquet

import jobqueue
from jobqueue.connection_manager import ConnectionManager



import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime

from typing import Callable, List

from psycopg import sql

import dmp.keras_interface.model_serialization as model_serialization
from dmp.task.experiment.training_experiment.training_epoch import TrainingEpoch
from dmp.postgres_interface.element.column import Column
from dmp.postgres_interface.element.table import Table
from dmp.postgres_interface.element.column_group import ColumnGroup

from dmp.util.butter_e_export import *

pd.options.display.max_seq_items = None

%load_ext autoreload
%autoreload 2
credentials = jobqueue.load_credentials("dmp")


2023-11-20 16:03:14.180828: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TODO: convert this into two steps:
    1) Get the ids of the runs we want to extract (in energy batches) ordered by experiment_id

          SELECT
                  {run}.{run_id}
          FROM
                  {run},
                  {job_status},
                  {job_data}
          WHERE TRUE
                  AND {run}.batch like {pattern}
                  AND {job_status}.id = {run}.run_id
                  AND {job_status}.id = {job_data}.id
              AND {job_status}.status = 2
          ORDER BY experiment_id, run_id;

    2) In parallel (using the multiprocessing lib), extract blocks of ids into a partitioned parquet file
      -> partition by the attributes we care about querying by (dataset, size, shape, depth)

              SELECT
                  {columns}
              FROM
                  {run},
                  {job_status},
                  {job_data}
              WHERE TRUE
                  AND {job_status}.id = {run}.run_id
                  AND {job_status}.id = {job_data}.id
                  AND {job_status}.status = 2
                  AND {run}.{run_id} IN ({ids})

        pool = multiprocessing.ProcessPool(multiprocessing.cpu_count())
        results = pool.uimap(download_chunk, chunks)
        for num_rows, chunk in results:
            num_stored += 1
            print(f"Stored {num_rows} in chunk {chunk}, {num_stored} / {len(chunks)}.")

    extra credit) extract butter data matching this as well into a new dataset
    extra credit) make a summary dataset that summarizes the quartiles of # epochs to reach target test loss levels
    

In [2]:
from psycopg import ClientCursor


print(f"run vars {vars(run)}")

columns = (
    run
    + ColumnGroup(*[c for c in job_status.columns if c.name != "id"])
    + job_data.command
)
print(columns.names)


def passthrough(row, index, value, column, data):
    data[column.name] = value


column_converters: List[Callable] = [passthrough for _ in columns]


def flatten_json(json_obj, destination=None, parent_key="", separator="_"):
    if isinstance(destination, dict):
        flattened = destination
    else:
        flattened = {}

    for key, value in json_obj.items():
        new_key = f"{parent_key}{separator}{key}" if parent_key else key
        if isinstance(value, dict):
            flattened.update(flatten_json(value, new_key, separator=separator))
        else:
            flattened[new_key] = value
    return flattened


column_converters[
    columns.get_index_of(job_data.command)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)
column_converters[
    columns.get_index_of(run.run_data)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)


def parquet_to_dataframe(row, index, value, column, data):
    with io.BytesIO(value) as buffer:
        data[column.name] = (
            pyarrow.parquet.read_table(pyarrow.PythonFile(buffer, mode="r"))
            .to_pandas()
            .sort_values(by="epoch")
        )


column_converters[columns.get_index_of(run.run_history)] = parquet_to_dataframe
column_converters[columns.get_index_of(run.run_extended_history)] = parquet_to_dataframe


dfs = []

run vars {'_name': 'run', '_columns': (), '_index': None}
('experiment_id', 'run_timestamp', 'run_id', 'job_id', 'seed', 'slurm_job_id', 'task_version', 'num_nodes', 'num_cpus', 'num_gpus', 'gpu_memory', 'host_name', 'batch', 'run_data', 'run_history', 'run_extended_history', 'queue', 'status', 'priority', 'start_time', 'update_time', 'worker', 'error_count', 'error', 'parent', 'command')
{Column(_name='experiment_id', type_name='uuid'): 0, Column(_name='run_timestamp', type_name='timestamp'): 1, Column(_name='run_id', type_name='uuid'): 2, Column(_name='job_id', type_name='uuid'): 3, Column(_name='seed', type_name='bigint'): 4, Column(_name='slurm_job_id', type_name='bigint'): 5, Column(_name='task_version', type_name='smallint'): 6, Column(_name='num_nodes', type_name='smallint'): 7, Column(_name='num_cpus', type_name='smallint'): 8, Column(_name='num_gpus', type_name='smallint'): 9, Column(_name='gpu_memory', type_name='integer'): 10, Column(_name='host_name', type_name='text'): 11,

In [3]:
import json
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import multiprocessing 
import tqdm

def get_ids(**kwargs):
    with ConnectionManager(credentials) as connection:
        query = psycopg.sql.SQL(
            """
                SELECT
                    {run}.run_id
                FROM
                    {run},
                    {job_status},
                    {job_data}
                WHERE TRUE
                    AND {run}.batch like {pattern}
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                    AND {job_status}.status = 2
                    AND {job_data}.command @> {json_data}::jsonb
                ORDER BY experiment_id, run_id;
            """ 
        ).format(
            run=run.identifier,
            job_status=job_status.identifier,
            job_data=job_data.identifier,
            pattern=sql.Literal("%energy%"),
            json_data=sql.Literal(get_json(**kwargs)
            )
        )

        with ClientCursor(connection) as c:
           c.mogrify(query)

        with connection.cursor(binary=True) as cursor:
            cursor.execute(query, binary=True)
            ids = cursor.fetchall()
            rows = []
            for id in ids:
                rows.append(str(id[0]))
            return rows
        
def get_json(**kwargs):
    # Given dictionary
    given_dict = {}
   
    # Update the given dictionary with user input
    given_dict.update(kwargs)

    # Convert the updated dictionary to JSON
    json_result = json.dumps(given_dict, indent=2)

    return json_result

In [4]:
def get_keys(d):
    keys = []
    for k, v in d.items():
        if isinstance(v, dict):
            keys.extend(get_keys(v))
        else:
            keys.append((k, v))
    return keys

def save_data(run_ids):
    with ConnectionManager(credentials) as connection:
        query = psycopg.sql.SQL(
            """
                SELECT
                    {columns}
                FROM
                    {run}, 
                    {job_status},
                    {job_data}
                WHERE
                	{job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                    AND {job_status}.status = 2
                    AND {run}.run_id IN ({run_ids});
            """ 
        ).format(
            columns=columns.columns_sql,
            run_ids=sql.SQL(', ').join(map(sql.Literal, run_ids)),
            run=run.identifier,
            job_data=job_data.identifier,
            job_status=job_status.identifier,
        )
       
        with ClientCursor(connection) as c:
            c.mogrify(query)
        
        with connection.cursor(binary=True) as cursor:
            cursor.execute(query, binary=True)
            rows = cursor.fetchall()
            
        # cast rows to a dataframe
        df = pd.DataFrame(rows, columns=columns.names)
        
        data = []
        data_names = []
        # iterate through the rows
        for index, row in df.iterrows():
            # get the keys from the command column
            keys = get_keys(row['command'])
            data_names = [key[0] for key in keys]
            data.append([key[1] for key in keys])
        
        # create a dataframe from the data
        df2 = pd.DataFrame(data, columns=data_names)
        # get intersection of columns
        intesection = list(set(df.columns) & set(df2.columns))
        # drop the intersection from the first dataframe
        df2 = df2.drop(intesection, axis=1)



        # merge the dataframes
        df = pd.concat([df, df2], axis=1)

        # convert to pyarrow table
        table = pa.Table.from_pandas(df)

        # write to distributed parquet file saved as ['name','depth','size','shape']
        pq.write_to_dataset(table, root_path='.', partition_cols=['name','shape','size','depth'])
        
        return df # delete the dataframe to free up memory


In [5]:
def combine_chunk_lists(lists, elements_per_chunk):
    combined = []
    for l in lists:
        combined.extend(l)
    # remove duplicates
    combined = list(set(combined))
    # split into chunks
    chunks = len(combined) // elements_per_chunk
    combined = np.array_split(combined, chunks)
    return combined

def parallel_save_data(chunks):
    for chunk in tqdm.tqdm(chunks):
        return save_data(chunk)
        

In [6]:

#  get some example data
ids_rec_depth5_sleep = get_ids(model={"shape":"rectangle", "depth":5},
                dataset={"name":"sleep"}
        )

ids_rec_depth6_sleep = get_ids(model={"shape":"rectangle", "depth":6},
                dataset={"name":"sleep"}
        )

print(f"Number of ids: {len(ids_rec_depth5_sleep)}")
print(f"Number of ids: {len(ids_rec_depth6_sleep)}")

batches = combine_chunk_lists([ids_rec_depth5_sleep, ids_rec_depth6_sleep], 10)

print(f"Number of batches: {len(batches)}")

parallel_save_data(batches)

Number of ids: 32
Number of ids: 30
Number of batches: 6


  0%|          | 0/6 [00:02<?, ?it/s]


,experiment_id,run_timestamp,run_id,job_id,seed,slurm_job_id,task_version,num_nodes,num_cpus,num_gpus,...,method,source,test_split,label_noise,validation_split,run_tags,class,learning_rate,precision,early_stopping
0,7a6907b6-ffbc-65e3-800a-5a2c76080b1a,2023-02-09 04:20:08.403282+00:00,0b1ae00c-8891-429f-bd78-3ec403102d95,0b1ae00c-8891-429f-bd78-3ec403102d95,1664220176,10392621,3,2,2,0,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
1,68111018-118a-6574-9b6f-5971db015b14,2023-02-09 05:23:55.316701+00:00,1b5e0585-d693-4931-8e41-9438f3285198,1b5e0585-d693-4931-8e41-9438f3285198,1663278490,10371433,3,1,2,2,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
2,d8e38f8a-cea6-f3f5-4f65-74b2691dc89a,2023-02-09 05:22:34.900706+00:00,40e564cd-58e9-46cb-a93d-e57ee34a22e6,40e564cd-58e9-46cb-a93d-e57ee34a22e6,1664218096,10343385,3,2,2,0,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
3,b46fa758-29f4-cab1-e32c-13fb90447326,2023-02-09 05:55:09.264404+00:00,46d86fca-1c78-4ff4-868d-5ecaabdd05d3,46d86fca-1c78-4ff4-868d-5ecaabdd05d3,1663278815,10373627,3,1,2,2,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
4,97ce666d-e4fa-3435-1377-36f0ba0c05c4,2023-02-09 05:39:10.793787+00:00,57a90cb7-fd69-4fbc-b74c-ec723d202f80,57a90cb7-fd69-4fbc-b74c-ec723d202f80,1664218681,10343233,3,2,2,0,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
5,a71de6b2-b936-ddd5-5bb3-cf9add27fd5d,2023-02-09 04:10:56.494323+00:00,7bb1f18f-da22-4370-8cfe-751c83113822,7bb1f18f-da22-4370-8cfe-751c83113822,1663280180,10296451,3,1,2,2,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
6,dfb84086-605b-20e0-af97-c753b71a4501,2023-02-09 04:11:35.690654+00:00,809ecbd3-e78d-4190-a4ec-ee1c5a2627b8,809ecbd3-e78d-4190-a4ec-ee1c5a2627b8,1664219981,10343277,3,2,2,0,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
7,379de7a7-b29a-8d38-acb6-0b4e9a4f135b,2023-02-09 05:15:54.907863+00:00,8472b423-9e1a-4d1a-9d87-14d6154c3016,8472b423-9e1a-4d1a-9d87-14d6154c3016,1663279270,10296466,3,1,2,2,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
8,b0d3321c-59da-5494-8f5f-ce63d204b693,2023-02-09 04:10:31.210916+00:00,aa6ae69b-706d-4ada-99ec-4acb1c200e85,aa6ae69b-706d-4ada-99ec-4acb1c200e85,1664219461,10343126,3,2,2,0,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
9,ab7ed21b-7897-e4d4-fd00-93bbd3085273,2023-02-09 04:10:29.002482+00:00,c411140c-ae39-413c-88da-742cac2714cb,c411140c-ae39-413c-88da-742cac2714cb,1664219396,10343251,3,2,2,0,...,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,float32,None
